In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import json
import ipdb
from overcooked_ai_py.mdp.overcooked_mdp import OvercookedState, PlayerState, OvercookedGridworld
%matplotlib inline
%config Completer.use_jedi = False

In [18]:
pkl_file = open("../human_aware_rl/data/human/anonymized/clean_train_trials.pkl", "rb")
df = pickle.load(pkl_file)

In [19]:
trial_id = df.workerid_num.unique()[0]
layout_id = df.layout_name.unique()[0]
trial_data = df[(df.workerid_num == trial_id) & (df.layout_name == layout_id)]
trial_data = trial_data.sort_values('cur_gameloop')

In [20]:
def get_state_diff(state, next_state, player, grid):
    # only getting the difference in held items
    # possible held items: onion, soup, dish
    # items = ["onion", "soup", "dish"]
    curr_item = None
    if "held_object" in state:
        curr_item = state["held_object"]["name"]
    next_item = None
    if "held_object" in next_state:
        next_item = next_state["held_object"]["name"]
    
    # possible high level actions: (encode with integer here, do one-hot encoding later)
    # 0: pick up onion
    # 1: pick pick up dish
    # 2: pick up soup (from counter)
    # 3: get cooked soup (from pot)
    # 4: put down onion
    # 5: put down dish
    # 6: put down soup (on counter)
    # 7: serve soup
    
    # I think the only way to do this mapping is manually
    if curr_item is None and next_item == "onion":
        return 0
    elif curr_item is None and next_item == "dish":
        return 1
    elif curr_item is None and next_item == "soup":
        return 2
    elif curr_item == "dish" and next_item == "soup":
        # TODO: check if this is how the soup item works
        return 3
    elif curr_item == "onion" and next_item is None:
        return 4
    elif curr_item == "dish" and next_item is None:
        return 5
    elif curr_item == "soup" and next_item is None:
        # TODO: split based on if soup was served or put on counter (somehow)
        return 6

def extract_hl_actions(data):
    # need to loop through and find the high level actions that the user picks, defined as:
    # grabbing an onion, grabbing a plate, grabbing a pot, putting down an object on counter,
    # putting an onion in to cook, grabbing a finished meal, or serving a finished meal
    data = data.sort_values('cur_gameloop')
    times = data.cur_gameloop
    grid = None
    for index, row in data.iterrows():
        if grid is None:
            grid = OvercookedGridworld.from_grid(eval(row["layout"]))
        # don't care about movement actions, we only need to split on "interact" actions
        action = eval(row["joint_action"])[0]
        state = eval(row["state"])
        next_state = eval(row["next_state"])
        if action == "INTERACT":
            get_state_diff(state, )

In [22]:
extract_hl_actions(trial_data)

> <ipython-input-20-82d31893e865>(47)extract_hl_actions()
     46 #     for t in times:
---> 47     for index, row in data.iterrows():
     48         # don't care about movement actions, we only need to split on "interact" actions

ipdb> grid = OvercookedGridworld.from_layout_name(row["layout"])
*** FileNotFoundError: [Errno 2] No such file or directory: "/Users/rapandya/dev/research/longterm_adapt/human_aware_rl/overcooked_ai/overcooked_ai_py/data/layouts/['XXXXXXXXX', 'O XSXOX S', 'X   P 1 X', 'X2  P   X', 'XXXDXDXXX'].layout"
ipdb> vercookedGridworld.from_grid(row["layout"])
*** NameError: name 'vercookedGridworld' is not defined
ipdb> grid = OvercookedGridworld.from_layout_name(row["layout"])
*** FileNotFoundError: [Errno 2] No such file or directory: "/Users/rapandya/dev/research/longterm_adapt/human_aware_rl/overcooked_ai/overcooked_ai_py/data/layouts/['XXXXXXXXX', 'O XSXOX S', 'X   P 1 X', 'X2  P   X', 'XXXDXDXXX'].layout"
ipdb> grid = OvercookedGridworld.from_grid(row["layout"

BdbQuit: 

In [9]:
x, y = trial_data[["cur_gameloop", "joint_action"]]